# Setup: LED's, RGB LED, Buttons

In [1]:
import threading
import asyncio
import time

from pynq.overlays.base import BaseOverlay

In [2]:
base = BaseOverlay("base.bit")
btns = base.btns_gpio
#leds = base.leds
#rgbled = base.rgbleds[4]
leds = [base.leds[0],base.leds[1],base.leds[2],base.leds[3],base.rgbleds[4]]

In [7]:
#testing each LED & buttons
'''
while True:
    for led in leds:
        for i in range(5):
            try:
                led.on(6) #6 is yellow
            except:
                led.on()
            time.sleep(1)
            led.off()
            time.sleep(1)
'''
cond=True

async def flash_leds():
    global cond
    while cond:
        for led in leds:
            for i in range(5):
                try:
                    led.on(6) #6 is yellow
                except:
                    led.on()
                await asyncio.sleep(0.2)
                led.off()
                await asyncio.sleep(0.2)

async def get_btns(_loop):
    global cond
    while True:
        await asyncio.sleep(0.01)
        for i in range(4):
            if btns[i].read() != 0: 
                cond = False
                _loop.stop()

loop = asyncio.new_event_loop()
loop.create_task(flash_leds())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()
print("Done.")

Done.


# Part 1: The 5 philosophers with sharing forks

In [2]:
base = BaseOverlay("base.bit")
btns = base.btns_gpio
leds = [base.leds[0],base.leds[1],base.leds[2],base.leds[3],base.rgbleds[4]]

In [15]:
#functions
async def blink(t, d, n):
    '''
    Function to blink the LEDs
    Params:
      t: number of times to blink the LED
      d: duration (in seconds) for the LED to be on/off
      n: index of the LED (0 to 4)
    '''
    for i in range(t):
        try:
            leds[n].on(6) #6 is yellow
        except:
            leds[n].on()
        await asyncio.sleep(d)
        led.off()
        await asyncio.sleep(d)
    leds[n].off()

async def worker_t(_locks, _fork_status, num):
    global stop_flag
    '''
    Worker function to try and acquire resource and blink the LED
    _l0: threading lock0 (fork0)
    _l1: threading lock1 (fork1)
    _l2: threading lock1 (fork2)
    _l3: threading lock1 (fork3)
    _l4: threading lock1 (fork4)
    num: index representing the LED and thread number.
    init: which resource this thread starts with (0 or 1)
    stop_flag: flag to temrinate thread
    '''
    print(f"Thread for philosopher {num} has started")
    while not stop_flag:
        if num == 0:
            left=4
            right=1
        elif num == 4:
            left=3
            right=0
        else:
            left=num-1
            right=num+1
        print(f"Philosopher {num} has fork {left} and fork {right}")
            
        print(f"left lock status: {_locks[left].locked()}\tright lock status: {_locks[left].locked()}")
        if _locks[left].locked() and _locks[right].locked(): #if forks are not free
            if _fork_status[left] == True and _fork_status[right] == True:
                await asyncio.sleep(0.1)
                print("fork are being used")
                #forks were used => release/NAP
                _locks[left].release()
                _locks[right].release()
                _fork_status[left] = False
                _fork_status[right] = False
                print(f"Philosopher {num} with fork{left} & fork{right} is NAPPING\n")
                await asyncio.sleep(0.5)
                blink(5, 0.1, num) #NAPPING blinks slower
            elif _fork_status[left] == False and _fork_status[right] == False:
                #forks were termed not used => set forks to being used - EATING
                await asyncio.sleep(0.1)
                print("fork are not being used")
                _fork_status[left] = True
                _fork_status[right] = True            
            else: #one of them is free
                await asyncio.sleep(0)
        elif not _locks[left].locked() and not _locks[right].locked(): #if both forks are free
            if _fork_status[left] == True and _fork_status[right] == True:
                #forks were used => was napping; Starve/wait for forks
                await asyncio.sleep(0.1)
                print("fork were used")
                _fork_status[left] = False
                _fork_status[right] = False
                print(f"Philosopher {num} with fork{left} & fork{right} is STARVING\n")
                leds[num].off()
            elif _fork_status[left] == False and _fork_status[right] == False:
                #forks are free; EAT - acquire locks
                #if forks are unused, acquire locks
                await asyncio.sleep(0.1)
                print("fork are free to use")
                _fork_status[left]=_locks[left].acquire(False)
                _fork_status[right]=_locks[right].acquire(False)
                print(f"Philosopher {num} with fork{left} & fork{right} is EATING\n")
                await asyncio.sleep(1)
                blink(50, 0.02, num) #EATING blinks faster
            else: #one of them is free
                 await asyncio.sleep(0) # yield
        else: #one or the other fork is free
             await asyncio.sleep(0) # yield

# Initialize and launch the threads
threads = []
locks = [threading.Lock(),threading.Lock(),threading.Lock(),threading.Lock(),threading.Lock()]
fork_status=[False, False, False, False, False]
stop_flag = False

async def get_btns(_loop):
    global stop_flag
    while True:
        await asyncio.sleep(0.01)
        for i in range(4):
            if btns[i].read() != 0: 
                stop_flag = True #set stop_flag (https://coderivers.org/blog/python-kill-a-thread/)
                for t in threads:
                    name = t.name
                    t.join()
                    print(f"{name} joined - waiting for thread to finish")
                _loop.stop()


async def philospher_init():
    for i in range(5): #thread made for each LED/philosopher
        print(f"Making thread for philosopher {i}")
        t = threading.Thread(target=worker_t, args=(locks, fork_status, i))
        print("thread made")
        threads.append(t)
        print("thread appended")
        t.start()
        print("thread started")
        
loop = asyncio.new_event_loop()
loop.create_task(philospher_init())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()
print("Done.")

TypeError: worker_t() missing 3 required positional arguments: '_locks', '_fork_status', and 'num'

# Part 2: Dining Philosophers with random integer range

In [17]:
from random import randint

base = BaseOverlay("base.bit")
btns = base.btns_gpio
leds = [base.leds[0],base.leds[1],base.leds[2],base.leds[3],base.rgbleds[4]]

In [ ]:
#functions
async def blink(t, d, n):
    '''
    Function to blink the LEDs
    Params:
      t: number of times to blink the LED
      d: duration (in seconds) for the LED to be on/off
      n: index of the LED (0 to 4)
    '''
    for i in range(t):
        try:
            leds[n].on(6) #6 is yellow
        except:
            leds[n].on()
        await asyncio.sleep(d)
        led.off()
        await asyncio.sleep(d)
    leds[n].off()

async def worker_t(_locks, _fork_status, num, _low, _high):
    global stop_flag
    '''
    Worker function to try and acquire resource and blink the LED
    _l0: threading lock0 (fork0)
    _l1: threading lock1 (fork1)
    _l2: threading lock1 (fork2)
    _l3: threading lock1 (fork3)
    _l4: threading lock1 (fork4)
    num: index representing the LED and thread number.
    init: which resource this thread starts with (0 or 1)
    stop_flag: flag to temrinate thread
    '''
    print(f"Thread for philosopher {num} has started")
    while not stop_flag:
        if num == 0:
            left=4
            right=1
        elif num == 4:
            left=3
            right=0
        else:
            left=num-1
            right=num+1
        print(f"Philosopher {num} has fork {left} and fork {right}")
            
        print(f"left lock status: {_locks[left].locked()}\tright lock status: {_locks[left].locked()}")
        if _locks[left].locked() and _locks[right].locked(): #if forks are not free
            if _fork_status[left] == True and _fork_status[right] == True:
                await asyncio.sleep(0.1)
                print("fork are being used")
                #forks were used => release/NAP
                _locks[left].release()
                _locks[right].release()
                _fork_status[left] = False
                _fork_status[right] = False
                print(f"Philosopher {num} with fork{left} & fork{right} is NAPPING\n")
                await asyncio.sleep(0.5)
                blink(5, 0.1, num) #NAPPING blinks slower
            elif _fork_status[left] == False and _fork_status[right] == False:
                #forks were termed not used => set forks to being used - EATING
                await asyncio.sleep(0.1)
                print("fork are not being used")
                _fork_status[left] = True
                _fork_status[right] = True            
            else: #one of them is free
                await asyncio.sleep(0)
        elif not _locks[left].locked() and not _locks[right].locked(): #if both forks are free
            if _fork_status[left] == True and _fork_status[right] == True:
                #forks were used => was napping; Starve/wait for forks
                await asyncio.sleep(0.1)
                print("fork were used")
                _fork_status[left] = False
                _fork_status[right] = False
                print(f"Philosopher {num} with fork{left} & fork{right} is STARVING\n")
                leds[num].off()
            elif _fork_status[left] == False and _fork_status[right] == False:
                #forks are free; EAT - acquire locks
                #if forks are unused, acquire locks
                await asyncio.sleep(0.1)
                print("fork are free to use")
                _fork_status[left]=_locks[left].acquire(False)
                _fork_status[right]=_locks[right].acquire(False)
                print(f"Philosopher {num} with fork{left} & fork{right} is EATING\n")
                await asyncio.sleep(1)
                blink(50, 0.02, num) #EATING blinks faster
            else: #one of them is free
                 await asyncio.sleep(0) # yield
        else: #one or the other fork is free
             await asyncio.sleep(0) # yield

# Initialize and launch the threads
threads = []
locks = [threading.Lock(),threading.Lock(),threading.Lock(),threading.Lock(),threading.Lock()]
fork_status=[False, False, False, False, False]
stop_flag = False

async def get_btns(_loop):
    global stop_flag
    while True:
        await asyncio.sleep(0.01)
        for i in range(4):
            if btns[i].read() != 0: 
                stop_flag = True #set stop_flag (https://coderivers.org/blog/python-kill-a-thread/)
                for t in threads:
                    name = t.name
                    t.join()
                    print(f"{name} joined - waiting for thread to finish")
                _loop.stop()


async def philospher_init():
    for i in range(5): #thread made for each LED/philosopher
        print(f"Making thread for philosopher {i}")
        low=randint(1,10)
        high=randint(50,100)
        t = threading.Thread(target=worker_t, args=(locks, fork_status, i, ))
        print("thread made")
        threads.append(t)
        print("thread appended")
        t.start()
        print("thread started")
        
loop = asyncio.new_event_loop()
loop.create_task(philospher_init())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()
print("Done.")